# Third assignment

Create a deep neural network that predicts the expected mean temperature (average of max and min for a given day) one day, one week, and one month in advance in Budapest.

Document the solution in detail (comments, readme.md) and upload it in Jupyter Notebook format. The solution should be easy to run.
Attach the training data to a separate CSV file, and have your solution display 3 predicted values for the following days:
* October 28
* November 3
* November 24

I used meteorological data (daily average temperatures) from 2017.01.01 to 2020.10.25, which I downloaded from https://meteostat.net/en/place/HU-YSS8 .

I built a 1D CNN, similar to that seen in exercise 7, which we used for sequential data regression. I used EarlyStopping for the training, with ModelCheckpoint to save the best model.
I chose 30 for patience parameter, because I saw that during model fitting, the resulting val_loss metric converged slowly with many local minimums far away from each other.
I also standardized the X_train, X_valid and X_test datasets, but not the target datasets. This way the predictions had the same unit of measurements as the original data.
( https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re )

For the forecast, I chose the following solution:

The constructed CNN works by taking window_size-length data sequences (say: data points from timestamp t1 to t20 with window_size = 20), and predicts the value right after the current sequence (so t21 in this example) based on it.
So if I take the last window_size-length sequence from the dataset as X_test, I get the predicted average temperature for the following day (October 26).
To get forecasts for an arbitrary-long time period I append the predicted value to the end of X_test with excluding the first element to get another window_size-length sequence.
Repeat this until the user gets the required number of predictions set in the num_of_forecast variable.


## Usage

Set the num_of_forecast variable according to how many days you want the forecast,
then simply run the script.

In [4]:
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

num_of_forecast = 30

# I chose the following parameters for the data split, model, training,...
valid_split = 0.2
num_filter = 32
filter_length = 5
# I use two months of data for one prediction
window_size = 60

if __name__ == "__main__":
    # read in the dataset from the relevant file and
    # reshape it for later use
    dataset = []
    with open('dataset.csv') as file:
        for row in file:
            row_split = row.rstrip().split(';')
            dataset.append(float(row_split[1]))
    dataset = np.array(dataset)
    dataset = np.atleast_2d(dataset).T

    # make window_size-length sequences
    # with targets starting from the element after the first sequence
    X = np.atleast_3d(
        np.array([dataset[start:start + window_size] for start in range(0, dataset.shape[0] - window_size + 1)]))
    Y = dataset[window_size:]

    # separate the last sequence as X_test and reshape it to
    # (num_samples, window_size, num_features) dimension required by the model
    X, X_test = X[:-1], X[-1]
    X_test = X_test.reshape((1, -1, 1))

    # split the data into train and validation databases
    num_samples = X.shape[0]
    train_size = int(num_samples * (1 - valid_split))
    X_train, Y_train = X[:train_size], Y[:train_size]
    X_valid, Y_valid = X[train_size:], Y[train_size:]

    # standardize X_train, X_valid and X_test
    train_mean = np.mean(X_train)
    train_std = np.std(X_train)

    X_train = (X_train - train_mean) / train_std
    X_valid = (X_valid - train_mean) / train_std
    X_test = (X_test - train_mean) / train_std

    # create the CNN
    model = Sequential()
    model.add(Conv1D(filters=num_filter, kernel_size=filter_length, activation='relu',
                     input_shape=(window_size, 1)))
    model.add(MaxPooling1D())
    model.add(Conv1D(filters=num_filter, kernel_size=filter_length, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])

    # fit the model with EarlyStopping and ModelCheckpoint as callbacks
    patience = 30
    early_stopping = EarlyStopping(patience=patience, verbose=1)
    checkpointer = ModelCheckpoint(filepath='model.hdf5', save_best_only=True, verbose=1)

    model.fit(X_train, Y_train, epochs=500, batch_size=16, validation_data=(X_valid, Y_valid), verbose=2,
              callbacks=[early_stopping, checkpointer])

    # load the model saved by ModelCheckpoint
    model = load_model('model.hdf5')

    # forecast future temperature values (num_of_forecast = 30)
    forecasted_values = []
    for i in range(0, num_of_forecast):
        # the prediction uses the same unit of measurement as the original data,
        # so I need to standardize it again after appending it to the forecasted_values
        preds = model.predict(X_test).flatten()
        forecasted_values.append(list(preds)[0])
        X_test = X_test[:, 1:, :]
        X_test = np.append(X_test, ((preds - train_mean) / train_std)).reshape((1, -1, 1))

    print(f"\nPredicted mean temperature for october 28:\t\t{forecasted_values[2]}")
    print(f"Predicted mean temperature for november 3:\t\t{forecasted_values[8]}")
    print(f"Predicted mean temperature for november 24:\t\t{forecasted_values[-1]}")



Epoch 1/1000

Epoch 00001: val_loss improved from inf to 36.05309, saving model to model.hdf5
67/67 - 0s - loss: 121.2504 - mae: 8.6392 - val_loss: 36.0531 - val_mae: 4.9044
Epoch 2/1000

Epoch 00002: val_loss improved from 36.05309 to 19.74185, saving model to model.hdf5
67/67 - 0s - loss: 30.2106 - mae: 4.4526 - val_loss: 19.7418 - val_mae: 3.6526
Epoch 3/1000

Epoch 00003: val_loss improved from 19.74185 to 15.23289, saving model to model.hdf5
67/67 - 0s - loss: 22.2331 - mae: 3.8527 - val_loss: 15.2329 - val_mae: 3.2097
Epoch 4/1000

Epoch 00004: val_loss improved from 15.23289 to 12.45829, saving model to model.hdf5
67/67 - 0s - loss: 16.5261 - mae: 3.3161 - val_loss: 12.4583 - val_mae: 2.8396
Epoch 5/1000

Epoch 00005: val_loss improved from 12.45829 to 11.07176, saving model to model.hdf5
67/67 - 0s - loss: 14.1767 - mae: 3.0664 - val_loss: 11.0718 - val_mae: 2.7447
Epoch 6/1000

Epoch 00006: val_loss improved from 11.07176 to 10.58314, saving model to model.hdf5
67/67 - 0s - lo